[논문 : Sequence to Sequence Learning with Neural Networks](https://arxiv.org/abs/1409.3215)
  
코드참고 : [케라스 블로그](https://keras.io/examples/nlp/lstm_seq2seq/)


> 진행 과정
1. 문장을 (encoder_input_data, decoder_input_data, decoder_target_data)의 3차원 배열로 변경. 
encoder_input_data : 3차원형태, 영어문장의 one-hot벡터화 형식의 데이터를 포함  
decoder_input_data : 3차원형태, 프랑스어문장의 one-hot벡터화 형식의 데이터를 포함  
decoder_target_data : decoder_input_data와 같으나 1타임스텝만큼 오프셋  
2. 기본 LSTM 기반의 Seq2Seq모델을 주어진 encoder, decoder input data로 decoder_target_data를 예측. teacher forcing을 사용한다.   
3. 모델이 잘 작동하는지 몇몇의 문장을 decode.





## 모듈 임포트 

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

## 데이터 다운로드  
원본 논문대로 영어 - 프랑스어 데이터 셋 다운로드

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

## 하이퍼 파라미터 설정

In [ ]:
batch_size = 64
epochs = 100
latent_dim = 256
num_samples = 10000
data_path = 'fra.txt'

## 데이터 준비

In [ ]:
# 데이터 벡터화
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path,"r",encoding='utf-8') as f:
  lines = f.read().split("\n")

for line in lines[: min(num_samples,len(lines) -1)]:
  input_text, target_text,_ = line.split("\t")
  # 탭을 시퀀스를 시작하는 문자로 사용. \n을 끝 시퀀스 문자로 사용
  target_text = '\t' + target_text + "\n"
  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data는 decoder_input_data에 비해서 1타임스텝 앞서있다.
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data는 시작문자를 포함하지 않으며, 1타임스텝 앞서간다.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 92
Max sequence length for inputs: 15
Max sequence length for outputs: 59


## 모델 생성

In [ ]:
# input sequence 정의
# Input의 경우 데이터의 모양을 model에 알려주는 역할을 한다. shape, batch_size등이 인수로 들어간다.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
# return_state : 출력과 함께 마지막 상태를 반환할지 여부. True로 하면 state를 2개 가져온다.
encoder = keras.layers.LSTM(latent_dim,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# encoder_outputs는 삭제. 
encoder_states = [state_h, state_c]

#디코더 설정, 초기상태로 encoder_states 사용
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

